In [ ]:
class Data():

    def __init__(self, valid_ratio, batch_size):
        
        """Downloads, splits and normalizes the data. Provides dataloaders for training"""
        
        mean_std = ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        
        transforms = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize(*mean_std)])

#         train_data = datasets.CIFAR10(root="../data",
#                             train=True,
#                             download=True)

#         train_data.data = torch.tensor(train_data.data)

#         channels = train_data.data.split(1, dim=-1)
#         channel_tensors = [channel.squeeze(-1) for channel in channels]

#         means = [z.float().mean() / 255 for z in channel_tensors]
#         stds = [z.float().std() / 255 for z in channel_tensors]

#         train_transforms = transforms.Compose([
#                             transforms.ToTensor(),
#                             transforms.Normalize(mean=means, std=stds)
#                                       ])

#         test_transforms = transforms.Compose([
#                                 transforms.ToTensor(),
#                                 transforms.Normalize(mean=means, std=stds)
#                                             ])

        train_data = datasets.CIFAR10(root="../data",
                                    train=True,
                                    download=True,
                                    transform=transforms)

        test_data = datasets.CIFAR10(root="../data",
                                train=False,
                                download=True,
                                transform=transforms)
        
        n_train_examples = int(len(train_data) * valid_ratio)
        n_valid_examples = len(train_data) - n_train_examples

        train_data, valid_data = data.random_split(train_data,
                                                [n_train_examples, n_valid_examples])

        valid_data = copy.deepcopy(valid_data)
        valid_data.dataset.transform = test_transforms

        self.train_loader = data.DataLoader(train_data,
                                        shuffle=True,
                                        batch_size=batch_size, num_workers=2)

        self.valid_loader = data.DataLoader(valid_data,
                                        batch_size=batch_size, num_workers=2)
        
        self.test_loader = data.DataLoader(test_data, batch_size=batch_size, num_workers=2)


    #     # downloads the data
    #     self.train_data = datasets.CIFAR10(root=ROOT, train=True, download=True)
    #     self.test_data = datasets.CIFAR10(root=ROOT, train=False,download=True)

    #     # splitting the data
    #     n_train_examples = int(len(self.train_data) * valid_ratio)
    #     n_valid_examples = len(self.train_data) - n_train_examples
    #     self.train_data, self.valid_data = data.random_split(self.train_data, [n_train_examples, n_valid_examples])
    #     self.valid_data = copy.deepcopy(self.valid_data)

    #     # normalizing the data

    #     t_mean, t_std = self.calculate_mean_std(self.train_data)
    #     train_transforms = transforms.Compose([
    #                         transforms.ToTensor(),
    #                         transforms.Normalize(mean=t_mean, std=t_std)])
    #     v_mean, v_std = self.calculate_mean_std(self.valid_data)
    #     valid_transforms = transforms.Compose([
    #                         transforms.ToTensor(),
    #                         transforms.Normalize(mean=v_mean, std=v_std)])
    #     te_mean, te_std = self.calculate_mean_std(self.test_data)
    #     test_transforms = transforms.Compose([
    #                         transforms.ToTensor(),
    #                         transforms.Normalize(mean=te_mean, std=te_std)])
        
    #     self.train_data.transform = train_transforms
    #     self.valid_data.transform = valid_transforms
    #     self.test_data.transform = test_transforms

    #     self.train_loader = data.DataLoader(self.train_data,
    #                              shuffle=True,
    #                              batch_size=batch_size, num_workers=2)
    #     self.valid_loader = data.DataLoader(self.valid_data,
    #                              batch_size=batch_size, num_workers=2)
        
    
    # def calculate_mean_std(self, dataset):
    #     """Calculates the mean and std of a dataset"""
    #     dataset.data = torch.tensor(dataset.data)
    #     channels = dataset.data.split(1, dim=-1)
    #     channel_tensors = [channel.squeeze(-1) for channel in channels]
    #     means = [z.float().mean() / 255 for z in channel_tensors]
    #     stds = [z.float().std() / 255 for z in channel_tensors]
    #     return means, stds

    class Inception(nn.Module):
    
    # c1--c5 are the number of output channels for each branch
    def __init__(self, c1, c2, c3, c4, c5, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # branch 1
        self.b1_1 = nn.LazyConv2d(c1, kernel_size=1)
        # branch 2
        self.b2_1 = nn.LazyConv2d(c2[0], kernel_size=1)
        self.b2_2 = nn.LazyConv2d(c2[1], kernel_size=3, padding=1)
        # branch 3
        self.b3_1 = nn.LazyConv2d(c3[0], kernel_size=1)
        self.b3_2 = nn.LazyConv2d(c3[1], kernel_size=5, padding=2)
        # branch 4 (added this branch to make the network wider)
        self.b4_1 = nn.LazyConv2d(c4[0], kernel_size=1)
        self.b4_2 = nn.LazyConv2d(c4[1], kernel_size=7, padding=3)
        # branch 5
        self.b5_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.b5_2 = nn.LazyConv2d(c5, kernel_size=1)
    
    # puts everything together
    def forward(self, x):
        b1 = F.relu(self.b1_1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(F.relu(self.b4_1(x))))
        b5 = F.relu(self.b5_2(self.b5_1(x)))
        return torch.cat((b1, b2, b3, b4, b5), dim=1)
    
    # inception block to see if we can weave in inception blocks into resnet for better results
    def inc(self):
        return nn.Sequential(Inception(32, (16, 32), (48, 64), (16, 32), 32),
                         Inception(64, (32, 64), (96, 128), (32, 64), 64),
                         nn.MaxPool2d(kernel_size=3, padding=1))